## Explore YT Music API

In [ ]:
import pandas as pd

from ytmusicapi import YTMusic
import ytmusicapi

TEST_PLAYLIST_ID ='PLXYeZ3VwLuLuZpwfs3GQQFvOuo7sg8IzF'

In [ ]:


yt = YTMusic("browser.json")
# playlistId = yt.create_playlist('test', 'test description')
search_results = yt.search('Oasis Wonderwall')
search_results
# yt.add_playlist_items(playlistId, [search_results[0]['videoId']])

In [ ]:
## user palylists
playlists = yt.get_library_playlists()
playlists[0]

In [ ]:
test_playlist = yt.get_playlist(TEST_PLAYLIST_ID)
print(len(test_playlist['tracks']))

In [ ]:
liked_songs = yt.get_liked_songs()
liked_songs['tracks'][0]

In [ ]:
liked_metadata = []

for song in liked_songs['tracks']:
    liked_metadata.append(yt.get_song(song['videoId']))


In [ ]:
liked_metadata[0]['videoDetails']

In [ ]:
## Choose low count liked songs:
keep_keys =['videoId','title','author','viewCount','lengthSeconds','publishDate']
liked_df =[]
for song in liked_metadata:
    # print(song['videoDetails'])
    liked_df.append({k:song['videoDetails'][k] for k in keep_keys})

liked_df = pd.DataFrame(liked_df)
# cast int viewCount
liked_df['viewCount'] = liked_df['viewCount'].astype(int)
liked_df = liked_df.sort_values('viewCount')
liked_df

## Put songs i like that aren't so popular on playlist

In [ ]:
rare_like = liked_df[liked_df['viewCount']<158030].videoId.tolist()
yt.add_playlist_items(playlistId=TEST_PLAYLIST_ID, videoIds=rare_like, )

## GEt history

In [ ]:
history = yt.get_history()
history

In [ ]:
## Remove any music that has been listened to recently in last 200 songs.
recent_listens = set([v['videoId'] for v in history])

# iterate over current playlist and remove any recent listens
removeable = [ v for v in test_playlist['tracks'] if v['videoId'] in recent_listens]
print(len(removeable), removeable)

yt.remove_playlist_items(playlistId=TEST_PLAYLIST_ID, videos=removeable, )

## for each one grab two recommended ones and add them to the list

In [ ]:
rare_like

In [ ]:
new_songs = []
for song in rare_like:
   new_songs.extend(yt.get_watch_playlist(videoId=song,limit=5, radio = True)['tracks'])

new_songs[:2]

In [ ]:
# drop any new_songs that are in recent
print(len(new_songs))

new_songs = [v for v in new_songs if v['videoId'] not in recent_listens]
print(len(new_songs), new_songs[:2])

In [ ]:
from time import sleep
print(len(new_songs), new_songs[:2])
new_metadata = []
for song in new_songs:
    new_metadata.append(yt.get_song(song['videoId']))
    sleep(0.3)
    

In [ ]:

new_df =[]
for song in new_metadata:
    # print(song['videoDetails'])
    new_df.append({k:song['videoDetails'][k] for k in keep_keys})

new_df = pd.DataFrame(new_df)
# cast int viewCount
new_df['viewCount'] = new_df['viewCount'].astype(int)
new_df = new_df.sort_values('viewCount')
new_df

In [ ]:
videoId_to_add = new_df[new_df['viewCount']<158030].videoId.tolist()

# Drop any in liked songs or in recent listens
videoId_to_add = [v for v in videoId_to_add if v not in recent_listens]
print(len(videoId_to_add), videoId_to_add[:2])
videoId_to_add = [v for v in videoId_to_add if v not in liked_songs]
print(len(videoId_to_add), videoId_to_add[:2])
# drop any already in test_playlist
test_playlist_ids = [v['videoId'] for v in test_playlist['tracks']]
videoId_to_add = [v for v in videoId_to_add if v not in test_playlist_ids]

In [ ]:
## Add to playlist
from time import sleep
for batch in range(0,len(videoId_to_add),5):
    print(batch)
    yt.add_playlist_items(playlistId=TEST_PLAYLIST_ID, videoIds=videoId_to_add[batch:batch+5], )
    sleep(1.3)